# Cut surface of paracetamol crystal

## By PyMatGen
The following example using PyMatGen does not work - Even though PyMatGen allows maintaining bonds, the cutting surface is still assumed to be, in general, a flat surface, which cannot be used to the case of Form I paracetamol

In [1]:
from pymatgen.io import cif
from pymatgen.core.surface import Slab, SlabGenerator, generate_all_slabs, Structure, Lattice, ReconstructionGenerator
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.core.structure import Structure, IStructure
from pymatgen.analysis.adsorption import *


f1bulk = cif.CifParser('F1-bulk.cif').get_structures(primitive=False)[0]
f1bulk = SpacegroupAnalyzer(f1bulk).get_conventional_standard_structure()
f1bulk.make_supercell([[1, 0, 0], 
                       [0, 4, 0], 
                       [0, 0, 1]])
bulk = f1bulk
interface = [0, 1, 0]
slab_thickness = 24.0
vac_thickness = 10.0
bond_list = {
    ('C', 'C'): 1.60,
    ('C', 'O'): 1.40,
    ('C', 'H'): 1.10,
    ('C', 'N'): 1.45,
    ('H', 'O'): 1.05,
    ('H', 'N'): 1.10
}

slab010 = SlabGenerator(bulk, 
                        miller_index=interface, 
                        min_slab_size=slab_thickness,
                        min_vacuum_size=vac_thickness, 
                        center_slab=True)
slab_struc = slab010.get_slabs(bonds = bond_list)
print(slab_struc)

[]


## A semi-automic solution

Paracetamol crystals are featured by hydrogen bond networks. The crystal is a vertical stack of hydrogen bond networks. Taking the Form I paracetamol as an example, the crystal is composed of 2 differently oriented hydrogen bond networks, which are stacked along the b axis. See the following figure:

![slab_struc-010.png](slab_struc-010.png)

A semi-automic solution is therefore proposed:

1. Analysis the periodic pattern and find the differently oriented surfaces. **The code does not recognize any rotation, mirror plane etc. to avoid drift in atomic coordinates**.  
2. In VESTA, from the bulk crystal, manually delete the atoms belonging to other networks and keep atoms of the desired network. The 2D periodicity should be kept.  
3. The 2D periodic hydrogen network is read in the following code blocks, as VASP POSCAR file written in Cartesian coordinates. The differently oriented hydrogen bond network is re-orginzed and arranged to generate the slab model. 

Note:

1. The relative distance between layer A and B cannot be established via this method. So independent A and B 2D periodic models should be generated from the same bulk model with the same origin. The distance between the same layer, i.e., A-A or B-B, equals the lattice parameter of the non-periodic dimension, which, in the case of Form I paracetamol, corresponds to *b*   
2. The sequence of layers is suggested to be 'bottom-up', i.e., lower layer first. The code cannot rearrange the sequence.

In [33]:
def get_lattice(filename):
    """
    Read lattice vectors. Return to a 3*3 numpy array.
    """
    import numpy as np

    file = open(filename, 'r')
    data = file.readlines()
    file.close()
    shrink = float(data[1].strip().split()[0])
    lattice = [data[2].strip().split(), 
               data[3].strip().split(), 
               data[4].strip().split()]
    lattice = np.array(lattice, dtype=float) * shrink
    
    return lattice

def get_atom(filename):
    """
    Read atomic species and coordinates. Only Cartesian coordinates are allowed.
    Return to species (natom*1 list) and coords (natom*3 array)
    """
    import re
    import numpy as np
    
    file = open(filename, 'r')
    data = file.readlines()
    file.close()
    
    if not re.match(r'^\s*C', data[7]):
        print('ERROR: Only Cartesian coordinates are premitted.')
        return
    
    labels = data[5].strip().split()
    count_species = data[6].strip().split()
    count_species = np.array(count_species, dtype=int)
    
    bg_line = 8
    species = []
    coords = []
    for idx_label, label in enumerate(labels):
        for idx_a in range(bg_line, bg_line + count_species[idx_label]):
            species.append(label)
            coords.append(data[idx_a].strip().split())
        
        bg_line += count_species[idx_label]
    
    coords = np.array(coords, dtype=float)
    
    return species, coords


def get_slab(file_list, index, nlayer, same_layer_space, thickness):
    """
    Expand lattice and combine atoms from different layers together to get the
    slab model.
    Note: The lattice used should be consistent. This is not checked by code.
    
    nlayer: Number of layers of the slab model
    index: Miller index of the slab surface
    same_layer_space: The length between same layers, in Angstrom
    thickness: Thickness  of the slab model: slab + vaccum layer, in Angstrom
    """
    from pymatgen.core.lattice import Lattice
    from pymatgen.core.structure import Structure
    import numpy as np
    
    species = []
    coords = []
    latt_mx = get_lattice(file_list[0])
    latt_pri = Lattice(latt_mx)
    for file in file_list:
        species.append(get_atom(file)[0])
        coords.append(get_atom(file)[1])
        
    # Get displacements in Cartesian coordinates
    space_unit = index * np.array(latt_pri.abc, dtype=float)
    space_unit = space_unit / np.linalg.norm(space_unit)
    space_xyz = np.dot(space_unit * same_layer_space / np.array(latt_pri.abc, dtype=float), latt_mx)
    # Displace atomic coordinates
    for idx_layer in range(len(file_list), nlayer):
        species.append(species[idx_layer % len(file_list)])
        space = space_xyz * (idx_layer // len(file_list))
        coords.append(coords[idx_layer % len(file_list)] + space)
    
    species_tot = []
    for i in range(nlayer):
        species_tot = species_tot + species[i]
        
    coords_tot = np.array(coords, dtype=float).reshape([-1, 3])
    
    # Extend lattice matrix
    thickness_abc = space_unit * thickness / np.array(latt_pri.abc, dtype=float)
    for i in range(3):
        if abs(thickness_abc[i]) < 1e-4:
            continue
            
        latt_mx[i,:] = latt_mx[i,:] * thickness_abc[i]
    
    lattice = Lattice(latt_mx)
    # Generate structure
    slab = Structure(lattice, species_tot, coords_tot, coords_are_cartesian=True)
    
    return slab
    

In [34]:
# Miller index of the slab
index = [0, 0, 1]
# Lattice parameter of the non-periodic dimension
same_layer_space = 12.667
# Stacking layers
layer = 4
# Independent layer files
layer_files = ['slab0-001-1.vasp', 'slab0-001-2.vasp']
# Thickness of the slab model: slab + vaccum layer Angstrom
thickness = 50
# Output CIF file name
outputname = 'slab5-001.cif'

from pymatgen.io.cif import CifWriter


slab = get_slab(layer_files, index, layer, same_layer_space, thickness)
CifWriter(slab).write_file(outputname)